### Урок 4. Парсинг HTML. XPath

* Выберите веб-сайт с табличными данными, который вас интересует.
* Напишите код Python, использующий библиотеку requests для отправки HTTP GET-запроса на сайт и получения HTML-содержимого страницы.
* Выполните парсинг содержимого HTML с помощью библиотеки lxml, чтобы извлечь данные из таблицы.
* Сохраните извлеченные данные в CSV-файл с помощью модуля csv.

Ваш код должен включать следующее:

- Строку агента пользователя в заголовке HTTP-запроса, чтобы имитировать веб-браузер и избежать блокировки сервером.
- Выражения XPath для выбора элементов данных таблицы и извлечения их содержимого.
- Обработка ошибок для случаев, когда данные не имеют ожидаемого формата.
- Комментарии для объяснения цели и логики кода.

Целью данной работы будет попытка получить данные по турнирам настольного тенниса от Лига-Про.

1. Пытаемся получить ответ с get-запроса по интересующему нас дню
2. Получить информацию по этому дню о предстоящих турнирах со списком участников
3. Сформировать из полученных данных датафрейм
4. Сохранить данных в табличном формате в файле csv

In [1]:
#!pip install lxml

In [2]:
# импортируем нужные библиотеки
from lxml import html
import requests
from pprint import pprint
import csv
import pandas as pd

In [3]:
url='https://tt.sport-liga.pro/'

# задаем user-agent
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'
    }
# в params надо указать интересующую нас дату в формате YYYY:MM:DD
# param = {
#     'year': '2024',
#     'month': '05',
#     'day': '7'
# }

url_tour = 'tours/'
session = requests.Session()

In [4]:
# просьба ввести данные пользователя из консоли (В РАЗРАБОТКЕ)
date = input('Введите дату в формате DD:MM:YYYY ')
day, month, year = date.split('.')
params = {
    'year': year,
    'month': month,
    'day': day
}

In [5]:
# делаем get-запрос
response = session.get(url+url_tour, params=params, headers=header)
# проверяем статус-код
if response.status_code < 400:
    # если да, печатаем статус код и представляем ответ в виде дерева html
    print(response.status_code)
    tree = html.fromstring(response.text)
else:
    print('Что-то пошло не так')

200


Необходимые данные находятся на странице в нижней таблице. Также для сбора фамилий участников турнира нам нужно переходить на его страницу (ее можно "достать" из таблицы.)

In [6]:
# находим нужную таблицу на странице
table = tree.xpath('//table[@class="table"]//tr')
# находим ссылки на переход к просмотру турнира
ref = tree.xpath('.//td[@class="tournament-name"]/a/@href')

Для объединения всех данных в один список будем проходиться циклом for сразу по двум спискам, для этого объединим их функцией zip

In [7]:
tournaments = []
for row, item in zip(table[1:], ref):
    # сделаем get-запрос на страницу турнира
    res = session.get(url+item, headers=header)
    # и получим дерево из его ответа
    tour = html.fromstring(res.text)
    # создадим словарик для каждого турнира
    tours_info = {}
    # из даты нам нужно только время начало турнира
    time = row.xpath('.//td[@class="tournament-date"]/text()')
    time = [el.split()[-1] for el in time][0]

    # из названия турнира мы возьмем
    tours = row.xpath('.//td[@class="tournament-name"]/a/text()')
    # название стола
    tables = [el.split()[1][:-1] for el in tours][0]
    # уровень лиги
    liga = [el.split()[3] for el in tours][0]

    # участников турнира получим из страницы турнира
    parties = tour.xpath('//td[@class="nowrap"]/a/text()')

    tours_info['date'] = date
    tours_info['time'] = time
    tours_info['tables'] = tables
    tours_info['liga'] = liga
    tours_info['parties'] = parties
    
    # добавим каждый словарь в список
    tournaments.append(tours_info)


In [8]:
pprint(tournaments)

[{'date': '22.5.24',
  'liga': '500-550',
  'parties': ['Илюхин Е', 'Обухов В', 'Кутузов О', 'Анисимов Е', 'Мамека М'],
  'tables': 'А6',
  'time': '23:45'},
 {'date': '22.5.24',
  'liga': '400-450',
  'parties': ['Беспалов Н', 'Федоров А', 'Слесарев В', 'Лбов Ю', 'Шершнев А'],
  'tables': 'А4',
  'time': '23:45'},
 {'date': '22.5.24',
  'liga': '450-500',
  'parties': ['Попов О', 'Зюганов О', 'Зеленский Р', 'Матонин А', 'Чернов В'],
  'tables': 'А5',
  'time': '23:30'},
 {'date': '22.5.24',
  'liga': '500-550.',
  'parties': ['Атоманчик К', 'Ляльков В', 'Галимов М', 'Дехтяренко А'],
  'tables': 'А15',
  'time': '20:00'},
 {'date': '22.5.24',
  'liga': '900-1000',
  'parties': ['Салкин Р', 'Фролов А И', 'Сташковский Г', 'Желубенков Ал-й'],
  'tables': 'А9',
  'time': '20:00'},
 {'date': '22.5.24',
  'liga': '350-400',
  'parties': ['Самоха Е', 'Амеличкин И', 'Мельников М', 'Лобанов М'],
  'tables': 'А3',
  'time': '20:00'},
 {'date': '22.5.24',
  'liga': '800-900',
  'parties': ['Хомен

Запишем полученную информацию в csv-файл

In [9]:
with open(f'./tournaments-{date}.csv', 'w', newline='', encoding='UTF-8') as f:
    writer = csv.DictWriter(f, fieldnames=tournaments[0].keys())
    writer.writeheader
    for row in tournaments:
        writer.writerow(row)
print('Данные успешно записаны')

Данные успешно записаны


Для наглядности представим наш список в виде датафрейма

In [10]:
df = pd.DataFrame(tournaments)
df

,date,time,tables,liga,parties
0,22.5.24,23:45,А6,500-550,"[Илюхин Е, Обухов В, Кутузов О, Анисимов Е, Ма..."
1,22.5.24,23:45,А4,400-450,"[Беспалов Н, Федоров А, Слесарев В, Лбов Ю, Ше..."
2,22.5.24,23:30,А5,450-500,"[Попов О, Зюганов О, Зеленский Р, Матонин А, Ч..."
3,22.5.24,20:00,А15,500-550.,"[Атоманчик К, Ляльков В, Галимов М, Дехтяренко А]"
4,22.5.24,20:00,А9,900-1000,"[Салкин Р, Фролов А И, Сташковский Г, Желубенк..."
5,22.5.24,20:00,А3,350-400,"[Самоха Е, Амеличкин И, Мельников М, Лобанов М]"
6,22.5.24,19:45,А6,800-900,"[Хоменко Р, Денисов А-м, Евлампиев В, Гришаев Е]"
7,22.5.24,19:45,А4,600-700,"[Ляличев Д, Крючков Е, Афанасьев М А, Гиревенк..."
8,22.5.24,19:30,А5,300-350,"[Чернявский М, Нурмухамедов М, Савченко А, Аку..."
9,22.5.24,16:00,А3,400-450,"[Слесарев В, Алексеев А, Шершнев А, Свердлов М]"


Вывод: В результате проделанной работы мы получаем датафрейм, содержащий нужную информацию. Этот датафрейм планируется использовать для подбора турнира с более предпочтительными игроками.

Также планируется исследовать каждый турнир на предмет статистики личных встреч между игроками

In [11]:
invalids = ['Скоблов', 'Мамека', 'Лукьянов', 'Сухарников', 'Кустов', 'Марейчев', 'Дворниченко']


In [12]:
#df = df[df['tables'] != 'А15']
df = df[df['time'] < '23:00'] 
df = df[df['time'] > '15:00']
df = df[df['tables'] != 'А15']
df

,date,time,tables,liga,parties
4,22.5.24,20:00,А9,900-1000,"[Салкин Р, Фролов А И, Сташковский Г, Желубенк..."
5,22.5.24,20:00,А3,350-400,"[Самоха Е, Амеличкин И, Мельников М, Лобанов М]"
6,22.5.24,19:45,А6,800-900,"[Хоменко Р, Денисов А-м, Евлампиев В, Гришаев Е]"
7,22.5.24,19:45,А4,600-700,"[Ляличев Д, Крючков Е, Афанасьев М А, Гиревенк..."
8,22.5.24,19:30,А5,300-350,"[Чернявский М, Нурмухамедов М, Савченко А, Аку..."
9,22.5.24,16:00,А3,400-450,"[Слесарев В, Алексеев А, Шершнев А, Свердлов М]"
11,22.5.24,15:45,А4,450-500,"[Минченков И, Алов Р, Новиков И, Чернов В]"
12,22.5.24,15:45,А6,250-300,"[Хургин В, Щавелев Н, Попроцкий А, Кузьмин А]"
13,22.5.24,15:30,А5,300-350,"[Куликов С, Яковлев А, Нурмухамедов М, Орлов О]"


In [13]:
# for el in invalids:
#     if el in df['parties']:
